In [1]:
from IPython import display

# Speech recognition tutorial

Our speech module is much smaller compared to other components such as perception. However, it plays an important role in enabling human-machine interaction.

This tutorial will focus on the implementation of speech recognition that is currently being used by the @Home team, which relies on [SpeechRecognition](https://github.com/Uberi/speech_recognition), which is a python library that supports several online and offline speech recognition engines and APIs. Additionally, this tutorial will also cover __Kaldi__ and how it was integrated into the speech recognition pipeline.

This tutorial depends on the following:

- [SpeechRecognition](https://github.com/Uberi/speech_recognition)
    - CMU's [PocketSphinx](https://pypi.org/project/pocketsphinx/) for offline speech recognition (Optionally, in case you do not want to use Google Speech) 
- [Kaldi](https://github.com/kaldi-asr/kaldi) - speech recognition toolkit
- [py-kaldi-asr](https://github.com/gooofy/py-kaldi-asr) - a python wrapper for Kaldi

The basic idea of speech recognition is to record speech in the form of a sound wave and convert it into a digital representation of the wave. Once we have this audio data, we can use it as input to a model to transcribe the audio into text. I will not delve into what tipes of models since it is not the focus of the tutorial, but keep in mind that there are several existing API's using SOA methods (Google for online and Kaldi for offline in this case). 

## List Contents:

- [Installation](#Installation)
    - [Requirements](#Requirements)
    - [SpeechRecognition](#SpeechRecognition)
    - [Kaldi](#Kaldi)
    - [py-kaldi-asr](#py-kaldi-asr)
    - [Kaldi pre-trained models](#Kaldi pre-trained models)
- [Python SpeechRecognition](#Python SpeechRecognition)
    - [Recognizer Class](#Recognizer Class)
    - [Working with a microphone](#Working with a microphone)
    - [Working Example](#Working Example)
- [mdr_speech_recognition](#mdr_speech_recognition)

## Installation

### Requirements

* **Python** d2.7, or 3.3+ (required)
* **PyAudio** 0.2.11+ (required only if you need to use microphone input, ``Microphone``)
* **PocketSphinx** (required only if you need to use the Sphinx recognizer, ``recognizer_instance.recognize_sphinx``)
* **Google API Client Library for Python** (required only if you need to use the Google Cloud Speech API, ``recognizer_instance.recognize_google_cloud``)
* **FLAC encoder** (required only if the system is not x86-based Windows/Linux/OS X)
* **wget** for additional non-kaldi packages.
* **Standard UNIX utilities**: bash, perl, awk, grep, and make.
* Linear-algebra package such as **ATLAS**, **CLAPACK**, or **OpenBLAS**.
* **Cython**


### SpeechRecognition

To install the SpeechRecognition API is as easy to type:

``pip install SpeechRecognition``

However, this API currently does not support Kaldi. For the purpose of this tutorial, install from here:

``git clone -b feature/py-kaldi-asr_support https://github.com/robertocaiwu/speech_recognition.gitt``

``cd speech_recognition && python setup.py install --user``


### Kaldi

1. ``git clone https://github.com/kaldi-asr/kaldi.git``
2. Navigate into the /Kaldi/tools folder and run ./extras/check dependencies.sh
to check for additional dependencies that are needed for installation.
3. After installing necessary dependencies, compile by running ``make -j #ofcores`` 
4. Navigate into the /Kaldi/src folder and ``run ./configure –shared`` 
5. ``make depend -j #ofcores`` 
6. ``make check -j #ofcores`` 
7. ``make -j #ofcores``

### py-kaldi-asr

``pip install py-kaldi-asr``

### Kaldi pre-trained models

You can find pre-trained models in various languages from the official [Kaldi documentation](https://kaldi-asr.org/models.html) page and in [zamia-speech](https://goofy.zamia.org/zamia-speech/asr-models/).

The specific one used in this tutorial is a model trained with 1200 hours of audio in English language which can  be downloaded from [here](#https://goofy.zamia.org/zamia-speech/asr-models/kaldi-generic-en-tdnn_f-r20190227.tar.xz). This model has decent background noise resistance and can also be used on phone recordings.


## Python SpeechRecognition

### Recognizer Class






In [10]:
import speech_recognition as sr

rec = sr.Recognizer()

The ``Recognizer`` creates a new instance which represents a collection of speech recognition functionality. This class supports different APIs for recognizing speech:


* ``recognize_sphinx()`` [CMU Sphinx](http://cmusphinx.sourceforge.net/wiki/) (works offline)
* ``recognize_google()`` Google Speech Recognition
* ``recognize_google_cloud()`` [Google Cloud Speech API](https://cloud.google.com/speech/)
* ``recognize_wit()`` [Wit.ai](https://wit.ai)
* ``recognize_bing()`` [Microsoft Bing Voice Recognition](https://www.microsoft.com/cognitive-services/en-us/speech-api)
* ``recognize_houndify()`` [Houndify API](https://houndify.com/)
* ``recognize_ibm()`` [IBM Speech to Text](http://www.ibm.com/smarterplanet/us/en/ibmwatson/developercloud/speech-to-text.html)

Out of the box, we can only use Google Speech Recognition. For the rest (with the exception of pocketsphinx), we need a username/password combination to use the online service.

<div class="alert alert-block alert-danger">
<b>Caution:</b>  The default key provided for Google Speech Recognition is for testing purposes only, and Google <b>may revoke it at any time</b>. It is not a good idea to use the Google Web Speech API in production. Even with a valid API key, you’ll be limited to only 50 requests per day, and there is no way to raise this quota. Fortunately, SpeechRecognition’s interface is nearly identical for each API, so what you learn today will be easy to translate to a real-world project. [source](https://realpython.com/python-speech-recognition/#the-recognizer-class)
</div>

However, if you installed the suggested package for this tutorial, the Recognizer class will also include:

* ``recognize_kaldi()`` [Kaldi](https://github.com/kaldi-asr/kaldi)

### Working with a microphone

In [11]:
mic = sr.Microphone()

The ``Microphone`` class creates an instance which represents a physical microphone on the computer. This class has three parameters:

- device_index
- sample_rate
- chuck_size

If ``device_index`` is unspecified or ``None``, the default microphone is used as the audio source. Otherwise, ``device_index`` should be the index of the device to use for audio input.

The microphone audio is recorded in chunks of ``chunk_size`` samples, at a rate of ``sample_rate`` samples per second (Hertz). If not specified, the value of ``sample_rate`` is determined automatically from the system's microphone settings.

Higher ``sample_rate`` values result in better audio quality, but also more bandwidth (and therefore, slower recognition). Additionally, some CPUs, such as those in older Raspberry Pi models, can't keep up if this value is too high.

Higher ``chunk_size`` values help avoid triggering on rapidly changing ambient noise, but also makes detection less sensitive. This value, generally, should be left at its default.

It is recommended to use a sample rate of 16,000 Hertz since the vast mayority of models are trained with audio data using this same sample rate.

### Working Example

In [12]:
model_directory='/home/rob/speech/data/models/tdnn-en'

mic = sr.Microphone()
rec = sr.Recognizer()
rec.load_kaldi_model(model_directory=model_directory)

loading model...
loading model... done.


In [14]:
# Start recording
with mic as source:
    rec.adjust_for_ambient_noise(source)
    print('Listening ...')
    audio = rec.listen(source)

    # Decode audio from microphone
    s = rec.recognize_kaldi(audio)
    print ("You said: ", s[0])

Listening ...
You said:  hello hello 


## mdr_speech_recognition

Currently in the mdr_speech_recognition package, only pocketsphinx and the default google speech are being used. Integrating the Kaldi wrapper is WIP. 

There is a reason for deciding to use both engines. By default we want to use Google Speech API as it provides with the best recognition accuracy. However, this requires the robot to always maintain a stable internet connection. In the case when it is not possible to connect to the internet, we need a fallback plan to still be able to recognize speech which in this case is Pocketsphinx.

This package is completely robot independent. 

In [ ]:
def recognize(self):
    with self.microphone as source:
        self.recognizer.adjust_for_ambient_noise(source)

    try:
        while not rospy.is_shutdown():
            rospy.loginfo('Listening...')
            with self.microphone as source:
                audio = self.recognizer.listen(source)
            rospy.loginfo('Got a sound; recognizing...')

            """
            Google over PocketSphinx: In case there is a internet connection
            use google, otherwise use pocketsphinx for speech recognition.
            """
            recognized_speech = ""
            if SpeechRecognizer.check_internet_connection():
                try:
                    recognized_speech = self.recognizer.recognize_google(audio)
                except sr.UnknownValueError:
                    rospy.logerr("Could not understand audio.")
                except sr.RequestError:
                    rospy.logerr("Could not request results.")
            else:
                try:
                    recognized_speech = self.recognizer.recognize_sphinx(audio)
                except sr.UnknownValueError:
                    rospy.logerr("Could not understand audio.")
                except sr.RequestError:
                    rospy.logerr("Could not request results.")

            if recognized_speech != "":
                rospy.loginfo("You said: " + recognized_speech)
                self.pub.publish(recognized_speech)

    except Exception as exc:
        rospy.logerr(exc)

The mdr_speech_recognition package is completely robot independent. You can run it by building the corresponding package 

``catkin build mdr_speech_recognition``

and then launching:

``roslaunch mdr_speech_recognition speech_recognition.launch ``

If you listen to the **/speech_recognizer** topic, you will get the text output of the speech recognition API.